In [1]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Dropout
from keras_tqdm import TQDMNotebookCallback
import keras.callbacks

from sqlalchemy import create_engine
import json
import pickle

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [55]:
version = 'v2'

# R  W  1
# R  P  1
# G  W  1
# G  P  1
# H  W  1
# H  P  1

# category = 'R'
# category = 'G'
category = 'H'

# bet_type = 'W'
bet_type = 'P'

In [56]:
engine = create_engine('sqlite:///../../data/race.db')
conn = engine.connect()
conn
dfoo = pd.read_sql_table('race', conn)
print('{} races loaded!'.format(len(dfoo)))

5927 races loaded!


In [57]:
dfo = dfoo.loc[dfoo['race_type'].isin([category])]
print('{} {} races!'.format(len(dfo), category))

1482 H races!


In [58]:
dfo['results'] = dfo['results_data'].map(json.loads)
dfo['runners'] = dfo['runners_data'].map(json.loads)
dfo.head(3)

/Users/jaco/.pyenv/versions/3.6.1/envs/tabby/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/jaco/.pyenv/versions/3.6.1/envs/tabby/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,id,meeting_name,location,venue_mnemonic,race_type,meeting_date,race_number,race_name,race_start_time,race_status,race_distance,results_data,num_runners,runners_data,quinella,exacta,trifecta,first_four,results,runners
36,37,CHARLTON,VIC,M,H,2017-10-01,1,DNR LOGISTICS 3YO PACE MS,2017-10-01 02:14:00,Paying,2100,"[[1], [5], [3], [4]]",6,"[{""runnerName"": ""TORQUE TOTHE STARZ"", ""runnerN...",3.3,15.6,36.1,57.3,"[[1], [5], [3], [4]]","[{'runnerName': 'TORQUE TOTHE STARZ', 'runnerN..."
37,38,CHARLTON,VIC,M,H,2017-10-01,2,AGISTMENT KILMORE PACE MS,2017-10-01 02:48:00,Paying,1609,"[[1], [2], [8], [7]]",8,"[{""runnerName"": ""SUBTLE EMBRACE"", ""runnerNumbe...",4.3,10.5,33.8,186.1,"[[1], [2], [8], [7]]","[{'runnerName': 'SUBTLE EMBRACE', 'runnerNumbe..."
38,39,CHARLTON,VIC,M,H,2017-10-01,3,VALE ROD GRANT TROT MS,2017-10-01 03:18:00,Paying,2100,"[[5], [1], [6], [2]]",10,"[{""runnerName"": ""SHESASUNDON"", ""runnerNumber"":...",6.5,14.0,91.7,249.9,"[[5], [1], [6], [2]]","[{'runnerName': 'SHESASUNDON', 'runnerNumber':..."


In [59]:
# extract runners from races
data_all = pd.DataFrame()
for ri, race in dfo.iterrows():
    data_all = data_all.append(race['runners'])    
data_all.tail(3)

,P_pred,P_prob,W_pred,W_prob,barrierNumber,claimAmount,finishingPosition,fixedOdds,has_odds,num_runners,...,place_rank,place_scaled,riderDriverName,runnerName,runnerNumber,trainerName,win_odds,win_perc,win_rank,win_scaled
5,0.069343,0.035674,0.016875,0.015591,0,0,4,"{'returnWin': 41, 'returnWinOpen': 41, 'return...",True,7,...,7.0,0.023448,MITCHELL MILLER,JUST A LEGEND,4,N C DYMOCK,52.1,0.019194,7.0,0.015841
6,0.085608,0.044041,0.029025,0.026816,0,0,0,"{'returnWin': 34, 'returnWinOpen': 26, 'return...",True,7,...,5.0,0.040860,JACK JUSTINS,SIMONDOU SU,5,W JUSTINS,30.9,0.032362,6.0,0.026709
7,0.998507,0.513688,0.769474,0.710905,0,0,1,"{'returnWin': 1.22, 'returnWinOpen': 1.28, 're...",True,7,...,1.0,0.343905,DYLAN EGERTON-GREE,CHIAROSCURO,6,M A LINDAU,1.2,0.833333,1.0,0.687752


In [60]:
# drop scratched
data = data_all.dropna(subset=['win_odds', 'place_odds'])
data = data[(data.win_odds > 0)]
data = data[(data.place_odds > 0)]
data.describe()

,P_pred,P_prob,W_pred,W_prob,barrierNumber,claimAmount,finishingPosition,num_runners,place_odds,place_perc,place_rank,place_scaled,runnerNumber,win_odds,win_perc,win_rank,win_scaled
count,14140.000000,14140.000000,1.414000e+04,14140.000000,14140.000000,14140.0,14140.000000,14140.000000,14140.000000,14140.000000,14140.000000,14140.000000,14140.000000,14140.000000,14140.000000,14140.000000,14140.000000
mean,0.299005,0.104438,1.053818e-01,0.104809,0.785290,0.0,1.001980,10.278218,5.372116,0.347840,5.553112,0.104809,5.847595,25.107793,0.124193,5.615983,0.104809
std,0.208452,0.077646,1.306888e-01,0.125287,3.703145,0.0,1.398218,2.976194,5.730837,0.243878,3.390921,0.077334,3.522598,30.105048,0.143558,3.397898,0.120848
min,0.000000,0.000000,7.347395e-09,0.000362,0.000000,0.0,0.000000,1.000000,1.000000,0.006667,1.000000,0.001886,1.000000,1.040000,0.001600,1.000000,0.001349
25%,0.126393,0.044373,2.438337e-02,0.024252,0.000000,0.0,0.000000,8.000000,2.000000,0.151515,3.000000,0.045106,3.000000,6.200000,0.030581,3.000000,0.025767
50%,0.248103,0.083950,5.766835e-02,0.058381,0.000000,0.0,0.000000,10.000000,3.500000,0.285714,5.000000,0.084249,5.000000,14.400000,0.069444,5.000000,0.058442
75%,0.446944,0.156925,1.275740e-01,0.129935,0.000000,0.0,2.000000,12.000000,6.600000,0.500000,8.000000,0.147176,8.000000,32.700000,0.161290,8.000000,0.135284
max,1.000000,0.623140,8.850901e-01,1.000000,80.000000,0.0,4.000000,20.000000,150.000000,1.000000,20.000000,1.000000,20.000000,625.000000,0.961538,20.000000,1.000000


In [61]:
# get label data
Y = data['finishingPosition']

if bet_type == 'W':
    Y = (Y == 1)
elif bet_type == 'P':
    Y = (Y == 1) | (Y == 2) | ((Y == 3) & (data['num_runners'] >= 8))

#print(data['finishingPosition'].head(10))
Y = Y.astype(int)
# Y.head(10)
Y.describe()

count    14140.000000
mean         0.296110
std          0.456556
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max          1.000000
dtype: float64

In [62]:
xn = data['num_runners']
xwp = data['win_perc']
xws = data['win_scaled']
xwr = data['win_rank']
xpp = data['place_perc']
xps = data['place_scaled']
xpr = data['place_rank']
X = pd.concat([xn, xwp, xws, xwr, xpp, xps, xpr], axis=1)
X.describe()

,num_runners,win_perc,win_scaled,win_rank,place_perc,place_scaled,place_rank
count,14140.000000,14140.000000,14140.000000,14140.000000,14140.000000,14140.000000,14140.000000
mean,10.278218,0.124193,0.104809,5.615983,0.347840,0.104809,5.553112
std,2.976194,0.143558,0.120848,3.397898,0.243878,0.077334,3.390921
min,1.000000,0.001600,0.001349,1.000000,0.006667,0.001886,1.000000
25%,8.000000,0.030581,0.025767,3.000000,0.151515,0.045106,3.000000
50%,10.000000,0.069444,0.058442,5.000000,0.285714,0.084249,5.000000
75%,12.000000,0.161290,0.135284,8.000000,0.500000,0.147176,8.000000
max,20.000000,0.961538,1.000000,20.000000,1.000000,1.000000,20.000000


In [63]:
# For a single-input model with 2 classes (binary classification):

n = len(X.columns)
print('input dimension = {}'.format(n))

epochs = 100
print('epochs = {}'.format(epochs))

layer_1 = 50
layer_2 = 50

tag = '{}{}x{}{}'.format(category, layer_1, layer_2, bet_type)
print('tag = {}'.format(tag))
file_name = '/Users/jaco/code/tabby/each_way/{}/models/{}.h5'.format(version, tag)

try:
    model = load_model(file_name)
    print('model loaded')
except OSError:
    model = Sequential()
    model.add(Dense(layer_1, activation='relu', input_dim=n))
    model.add(Dense(layer_2, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    print('model created')

input dimension = 7
epochs = 100
tag = H50x50P
model created


In [64]:
# Train the model, iterating on the data in batches of 32 samples
tqdm = TQDMNotebookCallback()

tbCallBack = keras.callbacks.TensorBoard(
    log_dir='/Users/jaco/code/tabby/each_way/{}/summary/{}'.format(version, tag), 
    histogram_freq=0,
    write_graph=True,
    write_images=True)

model.fit(
    X.as_matrix(), 
    Y.as_matrix(),
    validation_split=0.2,
    shuffle=True,
    epochs=epochs,
    batch_size=32,
    verbose=0,
    callbacks=[tqdm, tbCallBack])

# creates a HDF5 file
model.save(file_name)